In [24]:
'''
Reading the Original Dataset and calculating the total streams of a song and storing it in a new CSV
Below code is just used for data analysis and understaning the original dataset.
'''
import pandas as pd
import os
from IPython.display import clear_output
data_path = os.path.join('data/spotify_data.csv')
spambase = pd.read_csv(data_path, delimiter = ',')
def create_tables():
    spambase.head(10)
    # records = spambase[spambase['Region']==region]
    new_table = spambase.drop_duplicates(subset=['URL'],keep='first')
    new_table = new_table.dropna()
    print(len(new_table))
    new_table = new_table.loc[new_table['Date']<='2017-12-31']
    new_table['Total_Streams']=0
    new_table = new_table.reset_index()
    i=1
    for url in spambase['URL'].unique():
        clear_output(wait=True)
        print(round((i/len(spambase['URL'].unique())*100),3),"% ",i," Records Processed",flush=True)
        song = spambase[spambase['URL']==url]
        ind = new_table[new_table['URL']==url].index
        new_table.loc[new_table.index[ind],'Total_Streams']= song['Streams'].sum
        i+=1

    new_table = new_table.drop(['index','Date','Streams'], axis = 1)
    new_table.to_csv('spotify-processed-data.csv')
    
    return new_table
# df = create_tables() ##


In [ ]:
'''
spotify_data.csv -> Original dataset (~3.5 million rows)
results.csv -> Scraped audio features for all the songs in the data
'''
data_path = os.path.join('data/spotify_data.csv')
song_all_data = pd.read_csv(data_path, delimiter = ',')

data_path_extra = os.path.join('data/results.csv')
song_extra_data = pd.read_csv(data_path_extra, delimiter = ',')

In [ ]:
dicc = {}
day =1
for d in song_all_data['Date'].unique():          #Creating a dictionary to map all the days
    dicc[d] = day
    day+=1
    
def manageday(val):                            #This Function loops over the day after crossing 365 to 1
    if(val>365):
        return val-365
    else:
        return val
    
def date_to_day(val):                        #This Function is used to Map the Date column to the adjusted day
    return dicc[val]

In [ ]:
song_all_data['Day'] = song_all_data['Date'].map(date_to_day)   # Mapping the date to day
song_all_data['Day']=song_all_data['Day'].map(manageday)        # Handling the days greater than 365

In [ ]:
'''
Creating a supplementary dataset from ~3.5M rows(original dataset) to 74200 rows.
Each row contains URL of the song and it's corresponding rank list across all the region for a day
Aim: To find the median rank of a song on a particular day across all the regions.
'''

data_l = len(song_all_data)
i = 0
global_dict = {}
while i < data_l:
    clear_output(wait=True)
    print("Processing i :", i, flush=True)
    pos_list = []
    item = song_all_data.iloc[i]
    cur_url = item['URL']
    cur_day = item['Day']
    cur_pos = item['Position']
    if cur_url in global_dict:
        if cur_day in global_dict[cur_url]:
            pos_list = global_dict[cur_url][cur_day]
    pos_list.append(cur_pos)
    global_dict.setdefault(cur_url, {})[cur_day] = pos_list
    i += 1

print(len(global_dict))
## Saving results of global_dict in MedianRank.json
## filehandler = open('MedianRank.json', 'wb')
##pickle.dump(global_dict, filehandler)

In [7]:
import pickle 
filehandler = open('data/MedianRank.json', 'rb')  #loading the saved dictionary as the above cell took time and we saved the results
median = pickle.load(filehandler)            #So as to save time and not to re-run it every time


In [11]:
'''
Transforming global_dict into pandas dataframe
'''

data = pd.DataFrame()
i=1
for x in list(median.keys()):
    clear_output(wait=True)
    print(i," of ",len(list(median.keys())),"Records done ")
    daydic ={'URL': x,'Day': median[x].keys(),'MedianRank': median[x].values()}
    df = pd.DataFrame.from_dict(daydic)
    f =[data,df]
    data = pd.concat(f)
    i+=1
    

21738  of  21738 Records done 


In [14]:
data.head(10)

,URL,Day,MedianRank
0,https://open.spotify.com/track/3AEZUABDXNtecAO...,1,"[1, 3, 100, 1, 1, 4, 102, 4, 1, 113, 2, 1, 96,..."
1,https://open.spotify.com/track/3AEZUABDXNtecAO...,2,"[1, 2, 92, 1, 1, 3, 69, 3, 1, 94, 2, 1, 166, 9..."
2,https://open.spotify.com/track/3AEZUABDXNtecAO...,3,"[1, 2, 101, 1, 1, 2, 72, 3, 1, 97, 1, 1, 188, ..."
3,https://open.spotify.com/track/3AEZUABDXNtecAO...,4,"[1, 2, 100, 1, 1, 2, 68, 3, 1, 90, 1, 1, 186, ..."
4,https://open.spotify.com/track/3AEZUABDXNtecAO...,5,"[1, 2, 94, 1, 1, 2, 69, 1, 1, 82, 1, 1, 162, 9..."
5,https://open.spotify.com/track/3AEZUABDXNtecAO...,6,"[1, 2, 85, 1, 1, 2, 69, 2, 1, 78, 136, 1, 1, 1..."
6,https://open.spotify.com/track/3AEZUABDXNtecAO...,7,"[1, 2, 88, 1, 1, 2, 76, 1, 1, 83, 1, 1, 112, 2..."
7,https://open.spotify.com/track/3AEZUABDXNtecAO...,8,"[1, 2, 86, 1, 1, 2, 75, 1, 1, 88, 1, 1, 172, 1..."
8,https://open.spotify.com/track/3AEZUABDXNtecAO...,9,"[1, 2, 83, 1, 2, 2, 71, 1, 1, 67, 1, 1, 179, 1..."
9,https://open.spotify.com/track/3AEZUABDXNtecAO...,10,"[1, 1, 94, 1, 1, 1, 74, 1, 1, 81, 1, 1, 178, 1..."


In [20]:
'''
Below code is computing the Median Rank for the song from the position list across all regions
'''

import ast
import numpy as np
def med(val):
    return np.median(np.sort(val))

data['Median_Position'] = data['MedianRank'].map(med)

In [23]:
data.head(10)

,Unnamed: 0,URL,Day,MedianRank,Median_Position
0,0,https://open.spotify.com/track/3AEZUABDXNtecAO...,1,"[1, 3, 100, 1, 1, 4, 102, 4, 1, 113, 2, 1, 96,...",4.0
1,1,https://open.spotify.com/track/3AEZUABDXNtecAO...,2,"[1, 2, 92, 1, 1, 3, 69, 3, 1, 94, 2, 1, 166, 9...",3.0
2,2,https://open.spotify.com/track/3AEZUABDXNtecAO...,3,"[1, 2, 101, 1, 1, 2, 72, 3, 1, 97, 1, 1, 188, ...",2.0
3,3,https://open.spotify.com/track/3AEZUABDXNtecAO...,4,"[1, 2, 100, 1, 1, 2, 68, 3, 1, 90, 1, 1, 186, ...",2.0
4,4,https://open.spotify.com/track/3AEZUABDXNtecAO...,5,"[1, 2, 94, 1, 1, 2, 69, 1, 1, 82, 1, 1, 162, 9...",2.0
5,5,https://open.spotify.com/track/3AEZUABDXNtecAO...,6,"[1, 2, 85, 1, 1, 2, 69, 2, 1, 78, 136, 1, 1, 1...",2.0
6,6,https://open.spotify.com/track/3AEZUABDXNtecAO...,7,"[1, 2, 88, 1, 1, 2, 76, 1, 1, 83, 1, 1, 112, 2...",2.0
7,7,https://open.spotify.com/track/3AEZUABDXNtecAO...,8,"[1, 2, 86, 1, 1, 2, 75, 1, 1, 88, 1, 1, 172, 1...",2.0
8,8,https://open.spotify.com/track/3AEZUABDXNtecAO...,9,"[1, 2, 83, 1, 2, 2, 71, 1, 1, 67, 1, 1, 179, 1...",2.0
9,9,https://open.spotify.com/track/3AEZUABDXNtecAO...,10,"[1, 1, 94, 1, 1, 1, 74, 1, 1, 81, 1, 1, 178, 1...",1.0


In [ ]:
df_med =data.drop(['MedianRank'],axis=1)
merged_med = song_all_data.merge(df_med,on=['Day','URL'])      #Merging the median position and song data

In [ ]:
'''
We are merging top 200 songs across all the regions for the whole year (given in original dataset).
'''

df = pd.DataFrame()
index = np.arange(1,201,1)
# print(index)
for day in song_all_data['Day'].unique():
    d =song_all_data.loc[(song_all_data['Day']==day)].groupby(['URL']).Streams.sum().to_dict()
    d = dict(sorted(d.items(), key=lambda item: item[1], reverse=True)[0:200])
    dic = {'URL':list(d.keys()),
    'Per_Day_Streams': list(d.values()),
    'Rank': index,
        'Day' : day}
    df1= pd.DataFrame.from_dict(dic)
    f = [df,df1]
    df = pd.concat(f)

In [ ]:
#merged_med.to_csv('MergedMedianRank.csv')  #Saving the CSV
merged = df.merge(merged_med,on=['Day','URL'])

In [ ]:
final= merged.drop_duplicates(subset=['Day','Rank'],keep='first') #Dropping the duplicate rows with on Day and Rank leaving us with the song on one region only
final=final.reset_index()  #Resetting the Index

In [ ]:
final = final.merge(song_extra_data, on='URL') #Merging the scraped audio features from results.csv based on the URL

In [ ]:
#final.to_csv("data/GoldenData.csv") # Saving the final Preprocessd Dataset